# Copy The Tar Data Files For Each Month(2019-01 To 2019-03) To The Input Data Dir & Untar Those Files.

In [1]:
from pathlib import Path
import tarfile
import shutil
import os

# Set The Path From Where Process Will Pick The Input Data Files
# It Must Be An Existing Path
srcDataDir = Path('../problem-statement-artifacts/data/').resolve()

# Set The Directory Paths
inputDataDir = Path('../data/indir/').resolve()
procDataDir = Path('../data/procdir/').resolve()
outDataDir = Path('../data/outdir/').resolve()



# If The Directory Already Exist
# Clear The inputdir,procdir And outdir Dir (In Real Life Scenario We May Want To Archive These Dirs ).
if inputDataDir.is_dir():
    shutil.rmtree(str(inputDataDir))
if procDataDir.is_dir():
    shutil.rmtree(str(procDataDir))
if outDataDir.is_dir():
    shutil.rmtree(str(outDataDir))

    
# Create The indir,procdir And outdir Directory If They Do Not Already Exists
Path('../data/indir').mkdir(parents=True, exist_ok=True)
Path('../data/procdir').mkdir(parents=True, exist_ok=True)
Path('../data/outdir').mkdir(parents=True, exist_ok=True)


# Print the Info
print('Source Data Artifacts Dir (This Is Our Source System From Where We Will \
 Pull The Raw Input Files): \n\t{}'.format(srcDataDir))

print('\nInput Data Dir (Contains Raw Input Files): \n\t{}'.format(inputDataDir))

print('\nData Proc Dir (Raw Files Are Oraganized Into Folders Which Will Be Used \
 For Processing These Files): \n\t{}'.format(procDataDir))

print('\nData Out Dir (Contains Final Formated Output Data Files Which Will be \
 Loaded Into Elastic Search): \n\t{}'.format(procDataDir))


# Copy The Tar Files To Input Dir
for file in [file for file in srcDataDir.glob('**/*.gz') if file.is_file()]:
    shutil.copy(str(file), str(inputDataDir))
    
# Untar the Files In Input Dir And Delete The Tar Files
for tfile in [file for file in inputDataDir.glob('**/*.gz') if file.is_file()]:
    tarfile.open(str(tfile)).extractall(str(inputDataDir))
    os.remove(str(tfile))

Source Data Artifacts Dir (This Is Our Source System From Where We Will  Pull The Raw Input Files): 
	/home/dixitm/myGit/myworkspace/assignment/problem-statement-artifacts/data

Input Data Dir (Contains Raw Input Files): 
	/home/dixitm/myGit/myworkspace/assignment/data/indir

Data Proc Dir (Raw Files Are Oraganized Into Folders Which Will Be Used  For Processing These Files): 
	/home/dixitm/myGit/myworkspace/assignment/data/procdir

Data Out Dir (Contains Final Formated Output Data Files Which Will be  Loaded Into Elastic Search): 
	/home/dixitm/myGit/myworkspace/assignment/data/procdir


# Check The Files In The indir

In [2]:
for path,dirs,files in os.walk(str(inputDataDir)):
    print('\n{}'.format(path))
    for f in files:
        print('\t{}'.format(f))


/home/dixitm/myGit/myworkspace/assignment/data/indir

/home/dixitm/myGit/myworkspace/assignment/data/indir/2019-03
	2019-03-essex-street.csv
	2019-03-north-wales-outcomes.csv
	2019-03-cumbria-street.csv
	2019-03-norfolk-street.csv
	2019-03-warwickshire-outcomes.csv
	2019-03-west-mercia-outcomes.csv
	2019-03-northern-ireland-street.csv
	2019-03-lincolnshire-outcomes.csv
	2019-03-south-yorkshire-street.csv
	2019-03-cambridgeshire-street.csv
	2019-03-metropolitan-outcomes.csv
	2019-03-durham-street.csv
	2019-03-durham-outcomes.csv
	2019-03-cleveland-street.csv
	2019-03-west-yorkshire-street.csv
	2019-03-dyfed-powys-street.csv
	2019-03-hampshire-outcomes.csv
	2019-03-thames-valley-street.csv
	2019-03-merseyside-street.csv
	2019-03-dyfed-powys-outcomes.csv
	2019-03-wiltshire-street.csv
	2019-03-sussex-outcomes.csv
	2019-03-lancashire-street.csv
	2019-03-merseyside-outcomes.csv
	2019-03-norfolk-outcomes.csv
	2019-03-hampshire-street.csv
	2019-03-west-mercia-street.csv
	2019-03-north-yorkshi

# Copy The Files Into The procdir And Group Files Of Different Months Into 2 Folders. One Folder street (contains all street files from all months) & Other Folder outcome(all outcome files from all months are placed here)


# As We Copy The Files We Also Keep A Running Total Of Count Of Number Of Records In Each Category (street & outcome) Of Files. We Will Use This Count Later To Reconcile The Record Count Once The Data Is Loaded Into The Pandas Dataframe


# We also add a column name Rownum containing the line number to each file. We will be using this Rownum to remove Duplicate Records on "Crime ID" which are present within a file.

In [3]:
# Create The Folders street & outcome In The Procdir
Path(str(procDataDir) + '/street/').mkdir(parents=True, exist_ok=True)
Path(str(procDataDir) + '/outcome/').mkdir(parents=True, exist_ok=True)

# Set The Path Info
procDirStreet=Path(str(procDataDir) + '/street/').resolve()
procDirOutcome=Path(str(procDataDir) + '/outcome/').resolve()

#Print The procdir Subdirtory Details
print('Path For All street Files : \n\t{}'.format(procDirStreet))
print('\nPath For All outcome Files : \n\t{}'.format(procDirOutcome))

# Capture Total Record In street & outcomes Files For Reconciliation
totalStreetRecCount=0
totalOutcomesRecCount=0

# Copy All The street Data Files From inputDataDir To procDirStreet After adding Rownum to each line
for file in [file for file in inputDataDir.glob('**/*-street.csv') if file.is_file()]:
    rownum=0
    with open(str(file), 'r') as recs:
        data = recs.readlines()
    
    with open(str(procDirStreet) + '/' +os.path.basename(file), 'w') as recs:
        for (rownum, line) in enumerate(data):
            if rownum == 0:
                recs.write('{},{}'.format('Rownum', line))
            else:
                recs.write('{},{}'.format(rownum, line))

    # Get number of lines in the present file
    curr_file_num_lines = rownum
    totalStreetRecCount += curr_file_num_lines

    
# Copy All The outcomes Data Files From inputDataDir To procDirOutcome
for file in [file for file in inputDataDir.glob('**/*-outcomes.csv') if file.is_file()]:
    rownum=0
    with open(str(file), 'r') as recs:
        data = recs.readlines()
    
    with open(str(procDirOutcome) + '/' +os.path.basename(file), 'w') as recs:
        for (rownum, line) in enumerate(data):
            if rownum == 0:
                recs.write('{},{}'.format('Rownum', line))
            else:
                recs.write('{},{}'.format(rownum, line))

    # Get number of lines in the present file
    curr_file_num_lines = rownum
    totalOutcomesRecCount += curr_file_num_lines
    
    
print('\n\nTotal street Record Count From All Files: {}'.format(totalStreetRecCount))
print('\nTotal outcomes Record Count From All Files: {}'.format(totalOutcomesRecCount))

Path For All street Files : 
	/home/dixitm/myGit/myworkspace/assignment/data/procdir/street

Path For All outcome Files : 
	/home/dixitm/myGit/myworkspace/assignment/data/procdir/outcome


Total street Record Count From All Files: 1578168

Total outcomes Record Count From All Files: 1192446


# Check The Files In The procdir

In [4]:
for path,dirs,files in os.walk(str(procDataDir)):
    print('\n{}'.format(path))
    for f in files:
        print('\t{}'.format(f))


/home/dixitm/myGit/myworkspace/assignment/data/procdir

/home/dixitm/myGit/myworkspace/assignment/data/procdir/street
	2019-03-essex-street.csv
	2019-02-city-of-london-street.csv
	2019-03-cumbria-street.csv
	2019-03-norfolk-street.csv
	2019-01-south-yorkshire-street.csv
	2019-02-northumbria-street.csv
	2019-02-avon-and-somerset-street.csv
	2019-01-northumbria-street.csv
	2019-02-north-yorkshire-street.csv
	2019-03-northern-ireland-street.csv
	2019-03-south-yorkshire-street.csv
	2019-03-cambridgeshire-street.csv
	2019-01-norfolk-street.csv
	2019-02-sussex-street.csv
	2019-02-south-wales-street.csv
	2019-03-durham-street.csv
	2019-03-cleveland-street.csv
	2019-01-merseyside-street.csv
	2019-03-west-yorkshire-street.csv
	2019-03-dyfed-powys-street.csv
	2019-03-thames-valley-street.csv
	2019-01-northern-ireland-street.csv
	2019-01-greater-manchester-street.csv
	2019-01-west-midlands-street.csv
	2019-02-staffordshire-street.csv
	2019-01-lancashire-street.csv
	2019-01-west-mercia-street.csv

# Load The File Data Into Pandas Dataframes

In [60]:
import pandas as pd

# List All the street And outcomes
streetfileList=[str(file) for file in procDirStreet.glob('**/*-street.csv') if file.is_file()]
outcomefileList=[str(file) for file in procDirOutcome.glob('**/*-outcomes.csv') if file.is_file()]
 
# Map function which will read the csv file and also add one more column named "District name" to 
# the output dataframe
def read_csv_with_districtname_from_file(inputfile):
    outdf=pd.read_csv(inputfile)
    filename=os.path.basename(inputfile)
    districtname=filename[8:].replace('-street.csv','').replace('-outcomes.csv','')
    outdf['District name']=districtname
    return outdf
   
    
# Load The Data Into The Pandas Dataframes
dfStreet = pd.concat(map(read_csv_with_districtname_from_file, streetfileList))
dfOutcomes = pd.concat(map(read_csv_with_districtname_from_file, outcomefileList))


# Keep Columns Which Are Required

In [61]:
print('street data column list before dropping extra cols: \n{}'.format(dfStreet.columns))
print('\noutcome data column list before dropping extra cols: \n{}'.format(dfOutcomes.columns))

dfStreet=dfStreet[['Rownum', 'Crime ID', 'Month', 'Longitude', 'Latitude', 'Crime type', 'Last outcome category', 'District name']]
dfOutcomes=dfOutcomes[['Rownum', 'Crime ID', 'Month', 'Longitude', 'Latitude', 'Outcome type', 'District name']]

print('\n\nstreet data column list after dropping extra cols: \n{}'.format(dfStreet.columns))
print('\noutcome data column list after dropping extra cols: \n{}'.format(dfOutcomes.columns))

street data column list before dropping extra cols: 
Index(['Rownum', 'Crime ID', 'Month', 'Reported by', 'Falls within',
       'Longitude', 'Latitude', 'Location', 'LSOA code', 'LSOA name',
       'Crime type', 'Last outcome category', 'Context', 'District name'],
      dtype='object')

outcome data column list before dropping extra cols: 
Index(['Rownum', 'Crime ID', 'Month', 'Reported by', 'Falls within',
       'Longitude', 'Latitude', 'Location', 'LSOA code', 'LSOA name',
       'Outcome type', 'District name'],
      dtype='object')


street data column list after dropping extra cols: 
Index(['Rownum', 'Crime ID', 'Month', 'Longitude', 'Latitude', 'Crime type',
       'Last outcome category', 'District name'],
      dtype='object')

outcome data column list after dropping extra cols: 
Index(['Rownum', 'Crime ID', 'Month', 'Longitude', 'Latitude', 'Outcome type',
       'District name'],
      dtype='object')


# Reconcile the Record Counts In street & outcomes Files With The Counts Loaded In The dataframes

In [62]:
if dfStreet.shape[0] == totalStreetRecCount:
    print('SUCCESS: Record Count Match Between dfStreet dataframe: {} \
& Total Street Records: {}'.format(dfStreet.shape[0],totalStreetRecCount))
else:
    print('\nFATAL: Record Count Mismatch Between dfStreet dataframe: {} \
& Total Street Records: {}'.format(dfStreet.shape[0],totalStreetRecCount))
    
if dfOutcomes.shape[0] == totalOutcomesRecCount:
    print('\nSUCCESS: Record Count Match Between dfStreet dataframe: {} \
& Total Street Records: {}'.format(dfOutcomes.shape[0],totalOutcomesRecCount))
else:
    print('\nFATAL: Record Count Mismatch Between dfStreet dataframe: {} \
& Total Street Records: {}'.format(dfOutcomes.shape[0],totalOutcomesRecCount)) 
    


SUCCESS: Record Count Match Between dfStreet dataframe: 1578168 & Total Street Records: 1578168

SUCCESS: Record Count Match Between dfStreet dataframe: 1192446 & Total Street Records: 1192446


# Apply Schema To The DataFrames

# Convert Month From string value: 2019-01 to integer: 201901

In [63]:
commonColunsInStreetAndOutcomes=[]

print('Columns In street Dataframe: {}'.format(dfStreet.columns.shape[0]))
print('-----------------------------')
for col in dfStreet.columns:
    if col in ['Latitude','Longitude']:
        dfStreet[col]=dfStreet[col].astype(float)
    elif col in ['Month','Rownum']:
        if 'int' not in str(dfStreet[col].dtype):
            dfStreet[col]=dfStreet[col].str.replace('-','').astype(int)
    else:
        dfStreet[col]=dfStreet[col].astype(str)
    print('Column Name: {}, Data Type: {}'.format(col,dfStreet[col].dtype))

    
print('\nColumns In outcome Dataframe: {}'.format(dfOutcomes.columns.shape[0]))
print('-----------------------------')
for col in dfOutcomes.columns:
    if col in ['Latitude','Longitude']:
        dfOutcomes[col]=dfOutcomes[col].astype(float)
    elif col in ['Month','Rownum']:
        if 'int' not in str(dfOutcomes[col].dtype):
            dfOutcomes[col]=dfOutcomes[col].str.replace('-','').astype(int)
    else:
        dfOutcomes[col]=dfOutcomes[col].astype(str)
    print('Column Name: {}, Data Type: {}'.format(col,dfOutcomes[col].dtype))
    if col in dfStreet.columns:
        commonColunsInStreetAndOutcomes.append(col)
        

print('\nCommon Columns In street & outcome Dataframe: {}'.format(len(commonColunsInStreetAndOutcomes)))
print('-----------------------------')
for col in commonColunsInStreetAndOutcomes:
    print(col)

Columns In street Dataframe: 8
-----------------------------
Column Name: Rownum, Data Type: int64
Column Name: Crime ID, Data Type: object
Column Name: Month, Data Type: int64
Column Name: Longitude, Data Type: float64
Column Name: Latitude, Data Type: float64
Column Name: Crime type, Data Type: object
Column Name: Last outcome category, Data Type: object
Column Name: District name, Data Type: object

Columns In outcome Dataframe: 7
-----------------------------
Column Name: Rownum, Data Type: int64
Column Name: Crime ID, Data Type: object
Column Name: Month, Data Type: int64
Column Name: Longitude, Data Type: float64
Column Name: Latitude, Data Type: float64
Column Name: Outcome type, Data Type: object
Column Name: District name, Data Type: object

Common Columns In street & outcome Dataframe: 6
-----------------------------
Rownum
Crime ID
Month
Longitude
Latitude
District name


# Peek Into The Data Loaded in Dataframes

In [64]:
print('Shape Of The street Dataframe: {}'.format(dfStreet.shape))
dfStreet.head()

Shape Of The street Dataframe: (1578168, 8)


,Rownum,Crime ID,Month,Longitude,Latitude,Crime type,Last outcome category,District name
0,1,95e304d9e476cb5611111d51a3e258298232ba5bfafccb...,201903,-0.986073,51.935250,Violence and sexual offences,Unable to prosecute suspect,essex
1,2,4bfda2e430113728d4683311c7bcafa6f92638e208f0c1...,201903,0.806772,51.967013,Burglary,Investigation complete; no suspect identified,essex
2,3,13c85cc749572b0aad9375f680b939de8db7414e0cc84a...,201903,0.887522,51.992391,Other theft,Investigation complete; no suspect identified,essex
3,4,f4741e818da6bed6b6060584b295c4096bfd0341d95158...,201903,0.873053,51.970504,Violence and sexual offences,Investigation complete; no suspect identified,essex
4,5,6327dda5672144ac58e84036d1a39477db4e6e0e180ec8...,201903,0.158232,51.553923,Burglary,Under investigation,essex


In [65]:
print('Shape Of The street Dataframe: {}'.format(dfOutcomes.shape))
dfOutcomes.head()

Shape Of The street Dataframe: (1192446, 7)


,Rownum,Crime ID,Month,Longitude,Latitude,Outcome type,District name
0,1,c2fd1e9d414ec69ce0b1c64d12eead0175062836d7833c...,201902,NaN,NaN,Offender given suspended prison sentence,thames-valley
1,2,3d53d898b6af04e4ccc6429f9539da122eb25ce0117ba1...,201902,NaN,NaN,Offender sent to prison,thames-valley
2,3,a08159d3c4ecb568e48e4cd752f608cd3db4bf21cdf830...,201902,NaN,NaN,Offender sent to prison,thames-valley
3,4,8ca408c4088ed308a2b964be2dbeca0dc3a8ccb1b4f9da...,201902,NaN,NaN,Offender sent to prison,thames-valley
4,5,435381746e98bc5702b776751daf7a432401f7e22e5d01...,201902,NaN,NaN,Defendant sent to Crown Court,thames-valley


# Clean the street & outcomes Dataframe by following the below stops:

# 1) Removing duplicates on 'Crime ID','District name'. We retain the record with the maximum value of Month and In case we have more than one entry for the same 'Crime ID','District name' In the same month then we retain the record which has highest line number in a given file.

# 2) Delete the records from both the datasets where 'Crime ID' is not available

# 3) Keep only those columns which are needed in the final output dataset

In [68]:
rmd_dfStreet=dfStreet.sort_values(['Crime ID','District name','Month','Rownum'])\
.drop_duplicates(['Crime ID'],'last')

rmd_clean_dfStreet=rmd_dfStreet[rmd_dfStreet['Crime ID'] != 'nan']

rmd_clean_dfStreet=rmd_clean_dfStreet[['Crime ID', 'District name', 'Latitude'\
                                       , 'Longitude', 'Crime type', 'Last outcome category']]

print('Shape Of street dataset Before remove duplicate & nan drop on Crime Id: {}'\
      .format(dfStreet.shape))
print('Shape Of street dataset After remove duplicate & nan drop on Crime Id: {}'\
      .format(rmd_clean_dfStreet.shape))

rmd_clean_dfStreet.head()

Shape Of street dataset Before remove duplicate & nan drop on Crime Id: (1578168, 8)
Shape Of street dataset After remove duplicate & nan drop on Crime Id: (1229828, 6)


,Crime ID,District name,Latitude,Longitude,Crime type,Last outcome category
251,00000983358084d4e70b1ca90be352591bf506a6efda82...,north-yorkshire,53.897553,-1.992842,Burglary,Under investigation
13296,000022121d7fe64bb8a4c44497571b4a4f71ddb4f50368...,avon-and-somerset,NaN,NaN,Violence and sexual offences,Unable to prosecute suspect
2891,000035d52fae4f276492b3b1599eecbf3f1f1279e26ffc...,essex,51.785544,0.618594,Other theft,Investigation complete; no suspect identified
809,0000444a0b278a28da9e0dcb866584d3883289f6455ae9...,west-midlands,52.539277,-1.886178,Vehicle crime,Investigation complete; no suspect identified
87322,0000447043b24b920901e592522376db5be84b85e3ebbd...,metropolitan,51.533896,-0.189401,Burglary,Under investigation


In [69]:
rmd_dfOutcomes=dfOutcomes.sort_values(['Crime ID','District name','Month','Rownum'])\
.drop_duplicates(['Crime ID'],'last')

rmd_clean_dfOutcomes=rmd_dfOutcomes[rmd_dfOutcomes['Crime ID'] != 'nan']

rmd_clean_dfOutcomes=rmd_clean_dfOutcomes[['Crime ID','Outcome type']]

print('Shape Of outcomes dataset Before remove duplicate & nan drop on Crime Id: {}'\
      .format(dfOutcomes.shape))
print('Shape Of outcomes dataset After remove duplicate & nan drop  on Crime Id: {}'\
      .format(rmd_clean_dfOutcomes.shape))

rmd_clean_dfOutcomes.head()

Shape Of outcomes dataset Before remove duplicate & nan drop on Crime Id: (1192446, 7)
Shape Of outcomes dataset After remove duplicate & nan drop  on Crime Id: (1091966, 2)


,Crime ID,Outcome type
4070,000022121d7fe64bb8a4c44497571b4a4f71ddb4f50368...,Unable to prosecute suspect
1425,000022c155a80d717523dee51b31e08e28cfaa760b467a...,Unable to prosecute suspect
10799,000035d52fae4f276492b3b1599eecbf3f1f1279e26ffc...,Investigation complete; no suspect identified
8826,0000444a0b278a28da9e0dcb866584d3883289f6455ae9...,Investigation complete; no suspect identified
23637,00004b0f856f8989c91e674b9167d8c1a1a8665af7f191...,Investigation complete; no suspect identified


# ROUGH

In [31]:

rmd_dfStreet[rmd_dfStreet['Crime ID']=='nan'].shape

#st=dfOutcomes.groupby(['Crime ID','District name','Month']).size().reset_index(name='counts')



(45, 14)

In [33]:
dd=rmd_dfStreet.groupby('Crime ID')['District name'].nunique()

Crime ID
00000983358084d4e70b1ca90be352591bf506a6efda820bfd313251786a8c12     1
000022121d7fe64bb8a4c44497571b4a4f71ddb4f50368af86f5b488318c8f9e     1
000035d52fae4f276492b3b1599eecbf3f1f1279e26ffc49f399efbfdc830e62     1
0000444a0b278a28da9e0dcb866584d3883289f6455ae94629fd82b7b0ad75dd     1
0000447043b24b920901e592522376db5be84b85e3ebbd942c89f1aaba3784e2     1
00004b0f856f8989c91e674b9167d8c1a1a8665af7f191fa6855d9248bb8adff     1
0000568a00da3ef027781a3671bbef3b5d5d346017716922e410f822b2bd1b4e     1
00008c8e1e3a80fd2d2b7168abf1f3f02466731d1c5612ab8ecacf75b93b7273     1
0000989c9e21d0e26026af257f71e9928d937e4c0e80fc7c08f45d8c3957c627     1
0000aa030f87607413e66cdc5dc624b7247841c1d52b61668adc47ef64ccbeda     1
0000d39f4482bea98410126edfd7f8e0c38553fe0c6656c5226d9c02d081b515     1
0000d9e997e74ca7001e372c2b45993cf7e66e7dfd93339c7afa0ea26cf77eb0     1
0000dd4f598ed08fc80bcfcdeeb15192eafc763ad4ca6d084202e392f2ff327d     1
0000ee9d37d34c95752b37d32519c177299ec3279d91c21c09d8df6c9f85a893    

In [190]:
#dfStreet.set_index('Crime ID').join(dfStreet,on='Crime ID')

new_df = pd.merge(dfStreet, dfOutcomes,  how='left', left_on=['Crime ID'], right_on = ['Crime ID'])

In [27]:
dfOutcomes[dfOutcomes['Crime ID'] == '001901b7da234af86d6130656db771439b46911ed6c3ea1722d92b68d320aad2']

,Rownum,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Outcome type,District name
23202,23203,001901b7da234af86d6130656db771439b46911ed6c3ea...,201902,West Yorkshire Police,West Yorkshire Police,-1.781348,53.580885,On or near Petrol Station,E01011186,Kirklees 058C,Suspect charged,west-yorkshire
23203,23204,001901b7da234af86d6130656db771439b46911ed6c3ea...,201902,West Yorkshire Police,West Yorkshire Police,-1.781348,53.580885,On or near Petrol Station,E01011186,Kirklees 058C,Suspect charged,west-yorkshire
25221,25222,001901b7da234af86d6130656db771439b46911ed6c3ea...,201903,West Yorkshire Police,West Yorkshire Police,-1.781348,53.580885,On or near Petrol Station,E01011186,Kirklees 058C,Court case unable to proceed,west-yorkshire
25670,25671,001901b7da234af86d6130656db771439b46911ed6c3ea...,201903,West Yorkshire Police,West Yorkshire Police,-1.781348,53.580885,On or near Petrol Station,E01011186,Kirklees 058C,Court case unable to proceed,west-yorkshire


In [203]:

st=dfOutcomes.groupby(['Crime ID','District name']).size().reset_index(name='counts')
st[st['counts'] > 1]

,Crime ID,District name,counts
18,0001303424624d1309063f1cf4240fba0c111371018ab0...,leicestershire,2
19,00013b3cb3835fdefb281d88a3ddfb435d88042876cb28...,metropolitan,3
36,0002bea4269f05a36328dc02c1bfef8b7c74b4ce72999e...,derbyshire,2
56,0003fdca29242c8359ed08aee3bdffaffbeb75dfd40bc2...,cheshire,2
84,0005a656c9c7571d1b065f28138ae0c803498660ea8c87...,metropolitan,2
108,00078222fc99e38aaac8e1bbad9b36a2bd95e98c5a50e3...,greater-manchester,2
134,000953688bdf8e2a934c382b89643bfc8660d2a0459d7d...,cheshire,2
145,0009d03e3c76b9dab1bdd4011008ee2e919821efb7d8fc...,metropolitan,3
191,000c0b5da5c91c0cad719945132a1d04d961d987f2a274...,metropolitan,2
198,000c4640f6d6a05686d33c6f02575614a4c5cd1d2a2fdb...,metropolitan,2


In [218]:
st=dfStreet.groupby(['Crime ID','District name','Month']).size().reset_index(name='counts')
st[st['counts'] > 1]

,Crime ID,District name,Month,counts
233,000c4640f6d6a05686d33c6f02575614a4c5cd1d2a2fdb...,metropolitan,2019-01,2
409,0014f5805d0677b6f0d39df0da49d530390fd3f8e228bf...,avon-and-somerset,2019-03,2
420,0015627bff13044c64b465616896335908f556ea33fd53...,metropolitan,2019-03,2
442,00166f9e5adc90b6bd3c5e33d2168b9a82f32bf8c05973...,derbyshire,2019-02,2
572,001c831109defe8034e97d0f4615a212e1c0d4c3f35fb0...,metropolitan,2019-01,2
801,00280739896fda2e279814aaf48ba3509eaa2d1f6cd8b6...,metropolitan,2019-03,6
846,002a7b803ec3128e93360733c67cffc8fd6bf6388be679...,metropolitan,2019-01,2
904,002d3691e4a78d6d5205965b097cec9d6bd1c0b708d597...,metropolitan,2019-03,2
1027,003404c119705cc813544ecca9b13b42c77df8756ad160...,metropolitan,2019-03,2
1157,003af7bfd8d48ee8c19095b01793ba6e257e820888db64...,metropolitan,2019-02,2


In [220]:
dfStreet[dfStreet['Crime ID'].str.contains('000c4640f6d6a05686d33c6f02575614a4c5cd1d2a2fdb')]

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,District name
37485,000c4640f6d6a05686d33c6f02575614a4c5cd1d2a2fdb...,2019-01,Metropolitan Police Service,Metropolitan Police Service,-0.338022,51.583062,On or near Welldon Crescent,E01002128,Harrow 020A,Public order,Awaiting court outcome,nan,metropolitan
37486,000c4640f6d6a05686d33c6f02575614a4c5cd1d2a2fdb...,2019-01,Metropolitan Police Service,Metropolitan Police Service,-0.338022,51.583062,On or near Welldon Crescent,E01002128,Harrow 020A,Public order,Under investigation,nan,metropolitan


In [221]:
dfStreet['Last outcome category'].unique()


array(['Unable to prosecute suspect',
       'Investigation complete; no suspect identified',
       'Under investigation', 'nan', 'Local resolution',
       'Offender sent to prison',
       'Offender given a drugs possession warning',
       'Awaiting court outcome', 'Offender given a caution',
       'Formal action is not in the public interest',
       'Action to be taken by another organisation',
       'Defendant sent to Crown Court',
       'Further investigation is not in the public interest',
       'Offender ordered to pay compensation',
       'Offender given community sentence',
       'Court case unable to proceed', 'Offender fined',
       'Offender otherwise dealt with',
       'Offender given conditional discharge',
       'Offender given penalty notice',
       'Offender given suspended prison sentence',
       'Defendant found not guilty', 'Offender deprived of property',
       'Offender given absolute discharge',
       'Suspect charged as part of another case',
   

In [21]:
rmd_dfStreet=dfStreet.sort_values(['Crime ID','District name','Month',\
            'Rownum']).drop_duplicates(['Crime ID','District name'],'last')

In [22]:
rmd_dfStreet[rmd_dfStreet['Crime ID'].str.contains('000c4640f6d6a05686d33c6f02575614a4c5cd1d2a2fdb')]

,Rownum,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,District name
37486,37487,000c4640f6d6a05686d33c6f02575614a4c5cd1d2a2fdb...,201901,Metropolitan Police Service,Metropolitan Police Service,-0.338022,51.583062,On or near Welldon Crescent,E01002128,Harrow 020A,Public order,Under investigation,nan,metropolitan
